In [5]:
using GraphPPL, ReactiveMP, Distributions
# using Plots

┌ Info: Precompiling ReactiveMP [a194aa59-28ba-4574-a09c-4a745416d6e3]
└ @ Base loading.jl:1662


### ET

In [61]:
function f(x)
    return x
end

function f_inv(x)
    return x
end

c = randn(2);

@model function NKS()
    y2 = datavar(Float64)
    c = zeros(2); c[1] = 1.0;

    # γ ~ Gamma(α=1.0, β=1.0)
    x ~ MvNormal(μ=zeros(2), Λ=diageye(2))
    z ~ f(x) where {meta=ET(inverse=f_inv)}
    y1 ~ Normal(μ=dot(z, c), σ²=1.0)
    y2 ~ Normal(μ=y1, σ²=0.5)
end

In [62]:
imarginals = (x = MvNormal(zeros(2), diageye(2)), )

result = inference(model = Model(NKS), data=(y2=1.0,), free_energy=false)

"in" = "in"


Inference results:
-----------------------------------------
y1 = NormalWeightedMeanPrecision{Float64}(xi=2.0, w=2.5)
x  = MvNormalWeightedMeanPrecision(
xi: [0.6666666666666665, 0.0]
Λ: [1.6666666666676...


In [63]:
function f(x)
    return x
end

c = randn(2);

@model function NKS()
    y2 = datavar(Float64)
    c = zeros(2); c[1] = 1.0;

    # γ ~ Gamma(α=1.0, β=1.0)
    x ~ MvNormal(μ=zeros(2), Λ=diageye(2))
    z ~ f(x) where {meta=ET()}
    y1 ~ Normal(μ=dot(z, c), σ²=1.0)
    y2 ~ Normal(μ=y1, σ²=0.5)
end

In [64]:
imarginals = (x = MvNormal(zeros(2), diageye(2)), )

result = inference(model = Model(NKS), data=(y2=1.0,), free_energy=false)

m_ins = (MvNormalMeanPrecision(
μ: [0.0, 0.0]
Λ: [1.0 0.0; 0.0 1.0]
)
,)
(A, b) = ([1.0 0.0; 0.0 1.0], [0.0, 0.0])
μ_fw_out = [0.0, 0.0]
(ξ_bw_inx, Λ_bw_inx) = ([0.239999999999712, 0.0], [-0.40000000000035996 0.0; 0.0 -1.000088900582341e-12])


Inference results:
-----------------------------------------
y1 = NormalWeightedMeanPrecision{Float64}(xi=2.0, w=2.5)
x  = MvNormalWeightedMeanPrecision(
xi: [0.239999999999712, 0.0]
Λ: [0.59999999999964...


In [65]:
function f(x, θ)
    return x .+ θ
end

function f_inv_x(x)
    return randn(length(θ))
end

function f_inv_θ(x)
    return randn(length(x))
end

c = randn(2);

@model function NKS()
    y2 = datavar(Float64)
    c = zeros(2); c[1] = 1.0;

    θ ~ MvNormal(μ=ones(2), Λ=diageye(2))
    x ~ MvNormal(μ=zeros(2), Λ=diageye(2))
    z ~ f(x, θ) where {meta=ET(inverse=[f_inv_x, f_inv_θ])}
    y1 ~ Normal(μ=dot(z, c), σ²=1.0)
    y2 ~ Normal(μ=y1, σ²=0.5)
end

In [66]:
imarginals = (x = MvNormal(zeros(2), diageye(2)), )

result = inference(model = Model(NKS), data=(y2=1.0,), free_energy=false)

(meta.inverse[k])(mean(q_ins)) = [0.29709361820956753, 0.2813229134630898, 0.1217867624078451, -1.470889878920559]


LoadError: MethodError: no method matching f_inv_θ(::ForwardDiff.Dual{ForwardDiff.Tag{ReactiveMP.var"#g_unpacked#789"{typeof(f_inv_θ)}, Float64}, Float64, 4}, ::ForwardDiff.Dual{ForwardDiff.Tag{ReactiveMP.var"#g_unpacked#789"{typeof(f_inv_θ)}, Float64}, Float64, 4}, ::ForwardDiff.Dual{ForwardDiff.Tag{ReactiveMP.var"#g_unpacked#789"{typeof(f_inv_θ)}, Float64}, Float64, 4}, ::ForwardDiff.Dual{ForwardDiff.Tag{ReactiveMP.var"#g_unpacked#789"{typeof(f_inv_θ)}, Float64}, Float64, 4})
[0mClosest candidates are:
[0m  f_inv_θ(::Any) at In[65]:9

In [67]:
function f(x, θ, ζ)
    return x .+ θ .+ ζ
end

c = randn(2);

@model function NKS()
    y2 = datavar(Float64)
    c = zeros(2); c[1] = 1.0;

    θ ~ MvNormal(μ=ones(2), Λ=diageye(2))
    ζ ~ MvNormal(μ=0.5ones(2), Λ=diageye(2))
    x ~ MvNormal(μ=zeros(2), Λ=diageye(2))
    z ~ f(x, θ, ζ) where {meta=ET()}
    y1 ~ Normal(μ=dot(z, c), σ²=1.0)
    y2 ~ Normal(μ=y1, σ²=0.5)
end

In [68]:
imarginals = (x = MvNormal(zeros(2), diageye(2)), )

result = inference(model = Model(NKS), data=(y2=1.0,), free_energy=false)

(ξ_bw_inx, Λ_bw_inx) = ([-0.26666666666704, -1.999955756559757e-12], [-0.40000000000036007 0.0; 0.0 -1.000088900582341e-12])
(ξ_bw_inx, Λ_bw_inx) = ([-0.4666666666672199, -2.5000002068509275e-12], [-0.40000000000035996 0.0; 0.0 -1.000088900582341e-12])
(ξ_bw_inx, Λ_bw_inx) = ([-0.06666666666685998, -1.4999113062670868e-12], [-0.40000000000035973 -0.0; -0.0 -1.000088900582341e-12])


Inference results:
-----------------------------------------
y1 = NormalWeightedMeanPrecision{Float64}(xi=2.375, w=2.25)
ζ  = MvNormalWeightedMeanPrecision(
xi: [0.23333333333296002, 0.49999999999800004]
Λ:...
θ  = MvNormalWeightedMeanPrecision(
xi: [0.5333333333327801, 0.9999999999975]
Λ: [0.5...
x  = MvNormalWeightedMeanPrecision(
xi: [-0.06666666666685998, -1.4999113062670868e-1...


### UT

In [82]:
function f(x)
    return x
end

function f_inv(x)
    return x
end

c = randn(2);

@model function NKS()
    y2 = datavar(Float64)
    c = zeros(2); c[1] = 1.0;

    # γ ~ Gamma(α=1.0, β=1.0)
    x ~ MvNormal(μ=zeros(2), Λ=diageye(2))
    z ~ f(x) where {meta=UT(inverse=f_inv)}
    y1 ~ Normal(μ=dot(z, c), σ²=1.0)
    y2 ~ Normal(μ=y1, σ²=0.5)
end

In [83]:
imarginals = (x = MvNormal(zeros(2), diageye(2)), )

result = inference(model = Model(NKS), data=(y2=1.0,), free_energy=false)

(μ_tilde, Σ_tilde, _) = unscentedStatistics(μ_fw_in1, Σ_fw_in1, f; alpha = meta.alpha, beta = meta.beta, kappa = meta.kappa) = ([0.0, 0.0], [1.0 0.0; 0.0 1.0], [1.0 0.0; 0.0 1.0])
"in" = "in"


Inference results:
-----------------------------------------
y1 = NormalWeightedMeanPrecision{Float64}(xi=2.0, w=2.5)
x  = MvNormalWeightedMeanPrecision(
xi: [0.6666666666676693, 0.0]
Λ: [1.6666666666676...


In [84]:
function f(x)
    return x
end

c = randn(2);

@model function NKS()
    y2 = datavar(Float64)
    c = zeros(2); c[1] = 1.0;

    x ~ MvNormal(μ=zeros(2), Λ=diageye(2))
    z ~ f(x) where {meta=UT()}
    y1 ~ Normal(μ=dot(z, c), σ²=1.0)
    y2 ~ Normal(μ=y1, σ²=0.5)
end

In [85]:
imarginals = (x = MvNormal(zeros(2), diageye(2)), )

result = inference(model = Model(NKS), data=(y2=1.0,), free_energy=false)

(μ_tilde, Σ_tilde, _) = unscentedStatistics(μ_fw_in1, Σ_fw_in1, f; alpha = meta.alpha, beta = meta.beta, kappa = meta.kappa) = ([0.0, 0.0], [1.0 0.0; 0.0 1.0], [1.0 0.0; 0.0 1.0])
(μ_fw_in, Σ_fw_in) = (μs_fw_in, Σs_fw_in) = ([0.0, 0.0], [1.0 -0.0; -0.0 1.0])


Inference results:
-----------------------------------------
y1 = NormalWeightedMeanPrecision{Float64}(xi=2.0, w=2.5)
x  = MvNormalWeightedMeanPrecision(
xi: [0.39999999999976, 0.0]
Λ: [2.666666666667666...


In [86]:
function f(x, θ)
    return x .+ θ
end

function f_inv_x(x)
    return randn(length(θ))
end

function f_inv_θ(x)
    return randn(length(x))
end

c = randn(2);

@model function NKS()
    y2 = datavar(Float64)
    c = zeros(2); c[1] = 1.0;

    θ ~ MvNormal(μ=ones(2), Λ=diageye(2))
    x ~ MvNormal(μ=zeros(2), Λ=diageye(2))
    z ~ f(x, θ) where {meta=UT(inverse=[f_inv_x, f_inv_θ])}
    y1 ~ Normal(μ=dot(z, c), σ²=1.0)
    y2 ~ Normal(μ=y1, σ²=0.5)
end

In [88]:
imarginals = (x = MvNormal(zeros(2), diageye(2)), )

result = inference(model = Model(NKS), data=(y2=1.0,), free_energy=false)

"why am i called" = "why am i called"
(meta.inverse[k])(mean(q_ins)) = [-0.4342689992586467, -0.07854997401452626, -1.3322495205728564, -0.8484738167897723]
(ms, Vs) = mean_cov(q_ins) = ([-4.286015986565416e-13, -9.999778782798783e-13, 0.9999999999995715, 0.999999999999], [1.6666666666676662 -0.0 0.6666666666676662 -0.0; -0.0 1.000000000001 -0.0 9.999778782818784e-13; 0.6666666666676662 -0.0 1.666666666667666 -0.0; -0.0 9.999778782818784e-13 -0.0 1.000000000001])


LoadError: DimensionMismatch: dimensions must match: a has dims (Base.OneTo(2),), b has dims (Base.OneTo(4),), mismatch at 1

In [94]:
function f(x, θ, ζ)
    return x .+ θ .+ ζ
end

c = randn(2);

@model function NKS()
    y2 = datavar(Float64)
    c = zeros(2); c[1] = 1.0;

    θ ~ MvNormal(μ=ones(2), Λ=diageye(2))
    ζ ~ MvNormal(μ=0.5ones(2), Λ=diageye(2))
    x ~ MvNormal(μ=zeros(2), Λ=diageye(2))
    z ~ f(x, θ, ζ) where {meta=UT()}
    y1 ~ Normal(μ=dot(z, c), σ²=1.0)
    y2 ~ Normal(μ=y1, σ²=0.5)
end

In [95]:
imarginals = (x = MvNormal(zeros(2), diageye(2)), )

result = inference(model = Model(NKS), data=(y2=1.0,), free_energy=false)

LoadError: DimensionMismatch: dimensions must match: a has dims (Base.OneTo(2),), b has dims (Base.OneTo(6),), mismatch at 1